# Proyecto 1: Corpus Textual de Datos Climáticos de Oaxaca
## Análisis Exploratorio Completo (EDA)

**Objetivo**: Procesamiento y análisis riguroso de corpus multiregional de datos climáticos históricos de Oaxaca, México (junio-diciembre 2025).

**Período**: 01/06/2025 - 31/12/2025 (214 días)  
**Ciudades**: 8 regiones de Oaxaca  
**Documentos**: 1,712 (8 × 214)  
**Palabras totales**: 61,816  

---

## 1. Importar Librerías y Cargar Corpus

In [ ]:
import json
import pandas as pd
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Configurar estilo
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Descargar recursos NLTK
nltk.download('punkt', quiet=True)
nltk.download('punkt_tab', quiet=True)
nltk.download('stopwords', quiet=True)

SPANISH_STOP = set(stopwords.words("spanish"))
DOMAIN_STOP = {"ciudad", "país", "clima", "temperatura", "humedad"}
ALL_STOP = SPANISH_STOP | DOMAIN_STOP

print("✓ Librerías importadas exitosamente")

### 1.1 Cargar corpus JSONL

In [ ]:
def load_jsonl(path):
    """Carga corpus en formato JSONL."""
    docs = []
    with open(path, encoding="utf-8") as f:
        for line_num, line in enumerate(f, 1):
            try:
                docs.append(json.loads(line))
            except json.JSONDecodeError as e:
                print(f"Warning: línea {line_num} inválida: {e}")
    return docs

# Cargar corpus
corpus_path = "data/corpus_weather.jsonl"
print(f"Cargando corpus desde: {corpus_path}")
docs = load_jsonl(corpus_path)
print(f"✓ {len(docs)} documentos cargados\n")

# Mostrar documento ejemplo
print("EJEMPLO DE DOCUMENTO:")
print("-" * 80)
example = docs[0]
print(f"ID: {example['id']}")
print(f"Ciudad: {example['city']} ({example['region']})")
print(f"Fecha: {example['date']}")
print(f"Texto:\n{example['text'][:150]}...\n")
print(f"Metadatos: Temp.máx={example['metadata']['temp_max_celsius']}°C, "
      f"Lluvia={example['metadata']['precipitation_mm']}mm")

## 2. Estadísticas Básicas del Corpus

In [ ]:
def basic_stats(docs):
    """Calcula estadísticas de longitud."""
    if not docs:
        return {"n_docs": 0}
    
    lengths = [len(d["text"].split()) for d in docs]
    return {
        "n_docs": len(docs),
        "total_words": sum(lengths),
        "min_len": min(lengths),
        "max_len": max(lengths),
        "mean_len": np.mean(lengths),
        "median_len": np.median(lengths),
        "std_len": np.std(lengths)
    }

stats = basic_stats(docs)

print("ESTADÍSTICAS BÁSICAS DEL CORPUS")
print("=" * 60)
for key, val in stats.items():
    if isinstance(val, float):
        print(f"  {key:20s}: {val:10.2f}")
    else:
        print(f"  {key:20s}: {val:10d}")
print("=" * 60)

## 3. Análisis por Ciudad y Región

In [ ]:
# Crear DataFrame para análisis
df = pd.DataFrame([{
    'id': d['id'],
    'city': d['city'],
    'region': d['region'],
    'date': d['date'],
    'temp_max': d['metadata']['temp_max_celsius'],
    'temp_min': d['metadata']['temp_min_celsius'],
    'precipitation': d['metadata']['precipitation_mm'],
    'wind_speed': d['metadata']['wind_speed_kmh'],
    'text_length': len(d['text'].split())
} for d in docs])

print("RESUMEN POR CIUDAD")
print("-" * 80)
city_summary = df.groupby('city').agg({
    'id': 'count',
    'temp_max': ['min', 'mean', 'max'],
    'precipitation': 'sum'
}).round(2)
city_summary.columns = ['Documentos', 'T.Máx Min', 'T.Máx Prom', 'T.Máx Max', 'Lluvia Total']
print(city_summary)

print("\nRESUMEN POR REGIÓN")
print("-" * 80)
region_summary = df.groupby('region').agg({
    'id': 'count',
    'temp_max': 'mean',
    'precipitation': 'sum'
}).round(2)
region_summary.columns = ['Documentos', 'Temp. Prom. (°C)', 'Lluvia Total (mm)']
print(region_summary)

## 4. Tokenización y Análisis Léxico

In [ ]:
def tokenize(text):
    """Tokeniza y normaliza (lowercasing)."""
    tokens = [t.lower() for t in word_tokenize(text, language="spanish") 
              if t.isalpha()]
    return tokens

def top_k_tokens(docs, k=20, remove_stop=True, remove_high_freq=False, high_freq_threshold=None):
    """Extrae los k tokens más frecuentes."""
    ctr = Counter()
    for d in docs:
        toks = tokenize(d["text"])
        if remove_stop:
            toks = [t for t in toks if t not in ALL_STOP]
        ctr.update(toks)
    
    if remove_high_freq and high_freq_threshold is not None:
        freqs = sorted(ctr.values())
        threshold_val = np.percentile(freqs, high_freq_threshold)
        ctr = Counter({token: count for token, count in ctr.items() 
                      if count <= threshold_val})
    
    return ctr.most_common(k)

print("TOKENIZACIÓN COMPLETADA")
print("="*60)
print(f"Método: NLTK word_tokenize (español)")
print(f"Normalización: lowercasing + filtro alfabético")
print(f"Stopwords: NLTK español + términos de dominio")

## 5. Top-20 Tokens (Análisis de Frecuencias)

In [ ]:
# Top-20 sin stopwords
print("\nTOP-20 TOKENS (SIN STOPWORDS)")
print("=" * 60)
top_all = top_k_tokens(docs, k=20, remove_stop=True)
for i, (token, freq) in enumerate(top_all, 1):
    pct = (freq / stats['total_words']) * 100
    print(f"{i:2d}. {token:20s} -> {freq:5d} ({pct:5.2f}%)")

# Top-20 sin stopwords ni alta frecuencia
print("\nTOP-20 TOKENS (SIN STOPWORDS NI ALTA FRECUENCIA >P90)")
print("=" * 60)
top_filtered = top_k_tokens(docs, k=20, remove_stop=True, 
                            remove_high_freq=True, high_freq_threshold=90)
for i, (token, freq) in enumerate(top_filtered, 1):
    pct = (freq / stats['total_words']) * 100
    print(f"{i:2d}. {token:20s} -> {freq:5d} ({pct:5.2f}%)")

## 6. Visualización: Distribución de Tokens

In [ ]:
# Gráfico de top-20 tokens
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Top-20 sin filtrado
tokens, freqs = zip(*top_all)
ax1.barh(range(len(tokens)), freqs, color='steelblue')
ax1.set_yticks(range(len(tokens)))
ax1.set_yticklabels(tokens)
ax1.invert_yaxis()
ax1.set_xlabel('Frecuencia')
ax1.set_title('Top-20 Tokens (Sin Stopwords)', fontsize=14, fontweight='bold')
ax1.grid(axis='x', alpha=0.3)

# Top-20 filtrado
tokens_f, freqs_f = zip(*top_filtered)
ax2.barh(range(len(tokens_f)), freqs_f, color='coral')
ax2.set_yticks(range(len(tokens_f)))
ax2.set_yticklabels(tokens_f)
ax2.invert_yaxis()
ax2.set_xlabel('Frecuencia')
ax2.set_title('Top-20 Tokens (Filtrado >P90)', fontsize=14, fontweight='bold')
ax2.grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.savefig('outputs/top_tokens_comparison.png', dpi=300, bbox_inches='tight')
plt.show()
print("✓ Gráfico guardado: outputs/top_tokens_comparison.png")

## 7. Nubes de Palabras Generadas

In [ ]:
# Mostrar nubes de palabras generadas
from IPython.display import Image, display

wordclouds = [
    ('outputs/wordcloud_sin_filtrado.png', 'Nube sin filtrado (Todas las palabras)'),
    ('outputs/wordcloud_sin_stopwords.png', 'Nube sin stopwords (Palabras significativas)'),
    ('outputs/wordcloud_filtrado.png', 'Nube filtrada (Sin stopwords ni alta frecuencia)')
]

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for idx, (path, title) in enumerate(wordclouds):
    img = Image(filename=path)
    axes[idx].imshow(img.data)
    axes[idx].set_title(title, fontsize=12, fontweight='bold')
    axes[idx].axis('off')

plt.tight_layout()
plt.show()

print("\nNUBES DE PALABRAS GENERADAS:")
print("=" * 60)
for path, title in wordclouds:
    print(f"  ✓ {title}")
    print(f"     -> {path}")

## 8. Análisis por Región Temporal

In [ ]:
# Análisis por mes
df['month'] = pd.to_datetime(df['date']).dt.month
df['month_name'] = pd.to_datetime(df['date']).dt.strftime('%B')

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Temperatura por mes
monthly_temp = df.groupby('month')[['temp_max', 'temp_min']].mean()
axes[0, 0].plot(monthly_temp.index, monthly_temp['temp_max'], marker='o', label='T. Máxima', linewidth=2)
axes[0, 0].plot(monthly_temp.index, monthly_temp['temp_min'], marker='s', label='T. Mínima', linewidth=2)
axes[0, 0].fill_between(monthly_temp.index, monthly_temp['temp_min'], monthly_temp['temp_max'], alpha=0.2)
axes[0, 0].set_xlabel('Mes')
axes[0, 0].set_ylabel('Temperatura (°C)')
axes[0, 0].set_title('Temperatura Promedio por Mes')
axes[0, 0].legend()
axes[0, 0].grid(alpha=0.3)

# Precipitación por mes
monthly_precip = df.groupby('month')['precipitation'].sum()
axes[0, 1].bar(monthly_precip.index, monthly_precip.values, color='steelblue', alpha=0.7)
axes[0, 1].set_xlabel('Mes')
axes[0, 1].set_ylabel('Precipitación Total (mm)')
axes[0, 1].set_title('Precipitación Acumulada por Mes')
axes[0, 1].grid(alpha=0.3)

# Viento por mes
monthly_wind = df.groupby('month')['wind_speed'].mean()
axes[1, 0].bar(monthly_wind.index, monthly_wind.values, color='coral', alpha=0.7)
axes[1, 0].set_xlabel('Mes')
axes[1, 0].set_ylabel('Velocidad Promedio (km/h)')
axes[1, 0].set_title('Velocidad del Viento Promedio por Mes')
axes[1, 0].grid(alpha=0.3)

# Documentos por ciudad
city_counts = df['city'].value_counts()
axes[1, 1].barh(city_counts.index, city_counts.values, color='green', alpha=0.7)
axes[1, 1].set_xlabel('Número de Documentos')
axes[1, 1].set_title('Distribución de Documentos por Ciudad')
axes[1, 1].grid(alpha=0.3)

plt.tight_layout()
plt.savefig('outputs/analisis_temporal.png', dpi=300, bbox_inches='tight')
plt.show()
print("✓ Gráfico guardado: outputs/analisis_temporal.png")

## 9. Métricas Léxicas Avanzadas

In [ ]:
# Vocabulario único sin stopwords
all_tokens = []
for doc in docs:
    toks = tokenize(doc['text'])
    toks = [t for t in toks if t not in ALL_STOP]
    all_tokens.extend(toks)

unique_vocab = set(all_tokens)
vocab_size = len(unique_vocab)
ttr = vocab_size / len(all_tokens)  # Type-Token Ratio
hapax = sum(1 for t in unique_vocab if all_tokens.count(t) == 1)

print("MÉTRICAS LÉXICAS")
print("=" * 60)
print(f"Tamaño de vocabulario único: {vocab_size} palabras")
print(f"Tokens totales: {len(all_tokens)} palabras")
print(f"Type-Token Ratio (TTR): {ttr:.4f}")
print(f"Hapax legomena (palabras únicas): {hapax} ({100*hapax/vocab_size:.1f}%)")
print("\nInterpretación:")
print(f"  - TTR bajo ({ttr:.4f}) indica corpus muy repetitivo")
  f"(causado por generación automática de texto)")
print(f"  - {hapax} palabras aparecen solo una vez")
print(f"  - Vocabulario concentrado en ~{vocab_size} palabras clave")